In [3]:
from itertools import combinations_with_replacement

from ehrhart_quasi_polynomial.ehrhart_piecewise import (
    PiecewiseEhrhartQuasiPolynomial as PEQP,
    create_polytope_from_matrix,
    secondary_fan,
    _process_fan_vectors,
    _compute_change_of_basis_matrices,
    _hat_denominator,
    _generate_cone_points)

from ehrhart_quasi_polynomial import *

In [4]:
from sage.calculus.var import var
from sage.functions.other import ceil, factorial
from sage.geometry.cone import Cone
from sage.geometry.polyhedron.constructor import Polyhedron
from sage.matrix.constructor import Matrix
from sage.modules.free_module_element import free_module_element
from sage.modules.free_quadratic_module_integer_symmetric import IntegralLattice
from sage.rings.polynomial.polynomial_ring_constructor import PolynomialRing
from sage.rings.rational_field import QQ
from sage.symbolic.ring import SR

In [5]:
ehr = lambda A, b: ehrhart_quasi_polynomial(create_polytope_from_matrix(A, b).Vrepresentation())
num_int_points = lambda A, b: len(create_polytope_from_matrix(A, b).integral_points())

In [6]:
def test_combinations(p, A, xrange=range(-10, 11)):
        for b in combinations_with_replacement(xrange, A.nrows()):
            expected = num_int_points(A, b)
            actual = p(b)
            if actual != expected:
                print(b, expected, actual, rounded)
        print("all points were tested")

In [7]:
A = Matrix([[-1, 0], [0, -1], [1, 1]])
p = PEQP(A)

In [8]:
p._cone_dicts[0]["polynomials"]

{(0,): 9/2*x^2 + 9/2*x + 1, (1,): 9/2*x^2 + 3/2*x, (2,): 9/2*x^2 - 3/2*x}

In [9]:
test_combinations(p, A)

all points were tested


In [10]:
B = Matrix([[-1, 0], [0, -1], [1, 1], [0, 1]])
q = PEQP(B)

In [11]:
q._cone_dicts[1]["lifts"]

{(0, 0): (0, 0, 0, 0),
 (0, 1): (-1, -1, 0, -1),
 (0, 2): (-2, -2, 0, -2),
 (0, 3): (-3, -3, 0, -3),
 (0, 4): (-4, -4, 0, -4),
 (1, 0): (1, 4, 1, 4),
 (1, 1): (0, 3, 1, 3),
 (1, 2): (-1, 2, 1, 2),
 (1, 3): (-2, 1, 1, 1),
 (1, 4): (-3, 0, 1, 0),
 (2, 0): (2, 8, 2, 8),
 (2, 1): (1, 7, 2, 7),
 (2, 2): (0, 6, 2, 6),
 (2, 3): (-1, 5, 2, 5),
 (2, 4): (-2, 4, 2, 4),
 (3, 0): (3, 12, 3, 12),
 (3, 1): (2, 11, 3, 11),
 (3, 2): (1, 10, 3, 10),
 (3, 3): (0, 9, 3, 9),
 (3, 4): (-1, 8, 3, 8),
 (4, 0): (4, 16, 4, 16),
 (4, 1): (3, 15, 4, 15),
 (4, 2): (2, 14, 4, 14),
 (4, 3): (1, 13, 4, 13),
 (4, 4): (0, 12, 4, 12)}

In [12]:
test_combinations(q, B)

all points were tested


In [13]:
C = Matrix([[-1, 0], [0, -1], [1, 2], [0, 1]])
r = PEQP(C)

In [14]:
test_combinations(r, C, range(-6, 7))

all points were tested


In [16]:
D = Matrix([[-1, 0, 0], [0, -1, 0], [0, 0, -1], [1, 0, 0], [0, 1, 0], [0, 0, 1]])
s = PEQP(D)

In [17]:
s._cone_dicts[0]["polynomials"]

{(0, 0, 0): 8*x0*x1*x2 + 4*x0*x1 + 4*x0*x2 + 4*x1*x2 + 2*x0 + 2*x1 + 2*x2 + 1,
 (0, 0, 1): 8*x0*x1*x2 + 4*x0*x1 + 4*x0*x2 + 2*x0,
 (0, 1, 0): 8*x0*x1*x2 + 4*x0*x1 + 4*x1*x2 + 2*x1,
 (0, 1, 1): 8*x0*x1*x2 + 4*x0*x1,
 (1, 0, 0): 8*x0*x1*x2 + 4*x0*x2 + 4*x1*x2 + 2*x2,
 (1, 0, 1): 8*x0*x1*x2 + 4*x0*x2,
 (1, 1, 0): 8*x0*x1*x2 + 4*x1*x2,
 (1, 1, 1): 8*x0*x1*x2}

In [19]:
test_combinations(s, D)

all points were tested


In [21]:
E = Matrix([[-1, 0, 0], [0, -1, 0], [0, 0, -1], [1, 1, 1]])
t = PEQP(E)

In [22]:
t._cone_dicts[0]["polynomials"]

{(0,): 32/3*x^3 + 16*x^2 + 22/3*x + 1,
 (1,): 32/3*x^3 + 8*x^2 + 4/3*x,
 (2,): 32/3*x^3 - 2/3*x,
 (3,): 32/3*x^3 - 8*x^2 + 4/3*x}

In [23]:
test_combinations(t, E)

all points were tested


In [24]:
ehr(E, t._rays[0])

QuasiPolynomialElement(Ring of Quasi-Polynomials over Rational Field, [[1], [22/3], [16], [32/3]])

In [ ]:
F = Matrix([[-1, 0], [0, -1], [1, 3], [0, 1]])
u = PEQP(F)